## Road Events Json to CSV
- Input: the `.json` file generated by the 511 API: https://api.511.org/traffic/wzdx?api_key=789a13cf-d2e0-46db-95f2-54c2611933c1
    - The input file should be stored under the folder "wzdx_511"
- Output: A `.csv` file with all the information, which can be import into MySQL Database
    - The output location is `road_events/road_events_511.csv`

In [2]:
import json
import csv
import pandas as pd
import os
from datetime import datetime

In [7]:
folder_path = 'wzdx_511'

In [8]:
# About 15 minutes' running time
file_names = []
csv_file_path = 'road_events/road_events_511_5.csv'

    # Define the fieldnames for the CSV
fieldnames = [
    'id', 'type', 'data_source_id', 'event_type', 'road_names',
    'direction', 'description', 'creation_date', 'update_date',
    'start_date', 'end_date', 'event_status', 'start_date_accuracy',
    'end_date_accuracy', 'beginning_accuracy', 'ending_accuracy',
    'location_method', 'vehicle_impact', 'beginning_cross_street',
    'ending_cross_street','type_name',  'are_workers_present', 'worker_presence_definition',
    'worker_presence_confidence' , 'worker_presence_last_confirmed_date', 'geometry_type', 'geometry_coordinates'
]

def str_to_tinyint(s):
    if s is True or s == "True":
        return 1
    elif s is False or s == "False":
        return 0
    else:
        return 'NULL'

def parse_datetime(date_str):
    try:
        return datetime.strptime(date_str,'%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d %H:%M:%S')
    except (TypeError, ValueError):
        return 'NULL'
    
# Function to check if a row already exists in the CSV
def row_exists(csv_file_path, row_id):
    try:
        with open(csv_file_path, mode='r', newline='') as file:
            reader = csv.DictReader(file)
            for existing_row in reader:
                if existing_row['id'] == row_id:
                    return True
    except FileNotFoundError:
        # File not found, meaning no rows exist yet
        return False
    return False

for file in os.listdir(folder_path):
    # Construct the full file path
    file_path = os.path.join(folder_path, file)
    with open(file_path, 'r') as file:
        geojson_data = json.load(file)
        # Define the CSV file path


    # Open the CSV file for writing
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        
        # Iterate through each feature in the GeoJSON file
        for feature in geojson_data['features']:
            flag = 1
            # Extract core details and flatten the data
            core_details = feature['properties']['core_details']
            properties = feature['properties']
            geometry = feature['geometry']
            row_id = feature['id']
            if 'worker_presence' not in feature['properties']:
                feature['properties']['worker_presence'] = 1
                # If you have a specific flag variable, set it here
                flag = 0 # Get through all the rows with worker_presence
            else:
                worker_presence_wzdx = feature['properties']['worker_presence']
            # Check if the row already exists in the CSV
            if row_exists(csv_file_path, row_id):
                # Skip the row if it already exists
                continue
            
            # Prepare the row to be written to the CSV
            if flag == 1:
                row = {
                    'id': feature['id'],
                    'type': feature['type'],
                    'data_source_id': core_details.get('data_source_id'),
                    'event_type': core_details.get('event_type'),
                    'road_names': ', '.join(core_details.get('road_names', [])),
                    'direction': core_details.get('direction'),
                    'description': core_details.get('description'),
                    'creation_date': parse_datetime(core_details.get('creation_date')),
                    'update_date': parse_datetime(core_details.get('update_date')),
                    'start_date': parse_datetime(properties.get('start_date')),
                    'end_date': parse_datetime(properties.get('end_date')),
                    'event_status': properties.get('event_status'),
                    'start_date_accuracy': properties.get('start_date_accuracy'),
                    'end_date_accuracy': properties.get('end_date_accuracy'),
                    'beginning_accuracy': properties.get('beginning_accuracy'),
                    'ending_accuracy': properties.get('ending_accuracy'),
                    'location_method': properties.get('location_method'),
                    'vehicle_impact': properties.get('vehicle_impact'),
                    'beginning_cross_street': properties.get('beginning_cross_street'),
                    'ending_cross_street': properties.get('ending_cross_street'),
                    'type_name': ', '.join([tw['type_name'] for tw in properties.get('types_of_work', [])]),
                    'are_workers_present': worker_presence_wzdx.get('are_workers_present'),
                    'worker_presence_definition': ', '.join(worker_presence_wzdx.get('definition', [])),
                    'worker_presence_confidence': worker_presence_wzdx.get('confidence'),
                    'worker_presence_last_confirmed_date': parse_datetime(worker_presence_wzdx.get('worker_presence_last_confirmed_date')),
                    'geometry_type': geometry['type'],
                    'geometry_coordinates': ', '.join([f"[{', '.join(map(str, coord))}]" for coord in geometry['coordinates']])
                }
            else:
                row = {
                    'id': feature['id'],
                    'type': feature['type'],
                    'data_source_id': core_details.get('data_source_id'),
                    'event_type': core_details.get('event_type'),
                    'road_names': ', '.join(core_details.get('road_names', [])),
                    'direction': core_details.get('direction'),
                    'description': core_details.get('description'),
                    'creation_date': parse_datetime(core_details.get('creation_date')),
                    'update_date': parse_datetime(core_details.get('update_date')),
                    'start_date': parse_datetime(properties.get('start_date')),
                    'end_date': parse_datetime(properties.get('end_date')),
                    'event_status': properties.get('event_status'),
                    'start_date_accuracy': properties.get('start_date_accuracy'),
                    'end_date_accuracy': properties.get('end_date_accuracy'),
                    'beginning_accuracy': properties.get('beginning_accuracy'),
                    'ending_accuracy': properties.get('ending_accuracy'),
                    'location_method': properties.get('location_method'),
                    'vehicle_impact': properties.get('vehicle_impact'),
                    'beginning_cross_street': properties.get('beginning_cross_street'),
                    'ending_cross_street': properties.get('ending_cross_street'),
                    'type_name': ', '.join([tw['type_name'] for tw in properties.get('types_of_work', [])]),
                    'are_workers_present': 'NULL',
                    'worker_presence_definition': 'NULL',
                    'worker_presence_confidence': 'NULL',
                    'worker_presence_last_confirmed_date': 'NULL'
                }
           
        
            # Write the row to the CSV file
            writer.writerow(row)


    


In [6]:
# Re-define the path for the re-uploaded road_events CSV
re_uploaded_road_events_csv_path = 'road_events/road_events_511_5.csv'

# Function to remove rows where the first columns match specific values
def remove_rows_with_specific_columns(csv_path, columns_to_match):
    temp_path = csv_path + ".tmp"
    with open(csv_path, 'r', newline='') as input_file, open(temp_path, 'w', newline='') as output_file:
        writer = csv.writer(output_file)
        first_row = True
        for row in csv.reader(input_file):
            # Check if the row's first columns match the specified values
            if first_row:
                writer.writerow(row)
                first_row = False
                continue
            if row[:len(columns_to_match)] != columns_to_match:
                writer.writerow(row)
    # Replace the original file with the modified temp file
    os.replace(temp_path, csv_path)

# Columns to match for deletion
columns_to_match = ['id', 'type', 'data_source_id']

# Call the function to remove the specified rows from the re-uploaded road_events.csv
remove_rows_with_specific_columns(re_uploaded_road_events_csv_path, columns_to_match)